In [14]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Brazilian Ecommerce Public Dataset Olist

You can find this data in <a>https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce/code</a>

## Load Data

In [15]:
orders = pd.read_csv("../olist/olist_orders_dataset.csv")
orders_item= pd.read_csv("..\olist\olist_order_items_dataset.csv")
produts = pd.read_csv("..\olist\olist_products_dataset.csv")

customers = pd.read_csv("..\olist\olist_customers_dataset.csv")
geolocation = pd.read_csv("..\olist\olist_geolocation_dataset.csv")
payment = pd.read_csv("../olist/olist_order_payments_dataset.csv")
review = pd.read_csv("..\olist\olist_order_reviews_dataset.csv")

sellers = pd.read_csv("..\olist\olist_sellers_dataset.csv")
translation = pd.read_csv("..\olist\product_category_name_translation.csv")


## Data Resume

### Orders

#### 1 Data Resume

In [16]:
orders.head(), orders.shape

(                           order_id                       customer_id  \
 0  e481f51cbdc54678b7cc49136f2d6af7  9ef432eb6251297304e76186b10a928d   
 1  53cdb2fc8bc7dce0b6741e2150273451  b0830fb4747a6c6d20dea0b8c802d7ef   
 2  47770eb9100c2d0c44946d9cf07ec65d  41ce2a54c0b03bf3443c3d931a367089   
 3  949d5b44dbf5de918fe9c16f97b45f8a  f88197465ea7920adcdbec7375364d82   
 4  ad21c59c0840e6cb83a9ceb5573f8159  8ab97904e6daea8866dbdbc4fb7aad2c   
 
   order_status order_purchase_timestamp    order_approved_at  \
 0    delivered      2017-10-02 10:56:33  2017-10-02 11:07:15   
 1    delivered      2018-07-24 20:41:37  2018-07-26 03:24:27   
 2    delivered      2018-08-08 08:38:49  2018-08-08 08:55:23   
 3    delivered      2017-11-18 19:28:06  2017-11-18 19:45:59   
 4    delivered      2018-02-13 21:18:39  2018-02-13 22:20:29   
 
   order_delivered_carrier_date order_delivered_customer_date  \
 0          2017-10-04 19:55:00           2017-10-10 21:25:13   
 1          2018-07-26 14:31:00 

In [17]:
orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'])
orders['order_approved_at'] = pd.to_datetime(orders['order_approved_at'])
orders['order_delivered_carrier_date'] = pd.to_datetime(orders['order_delivered_carrier_date'])
orders['order_delivered_customer_date'] = pd.to_datetime(orders['order_delivered_customer_date'])
orders['order_estimated_delivery_date'] = pd.to_datetime(orders['order_estimated_delivery_date'])

##### Variaveis Alvo

* is_late (binary) - order_delivered_customer_data > order_estimated_delivery_date
* delivery_delay_days (numeric) - days of delay in delivery
* purchase_to_order_days
* order_to_shipment_days
* shipment_to_delivery_days
* delivery_to_estimated_delivery_days


### EDA

In [21]:
orders['order_status'].value_counts().reset_index()


,order_status,count
0,delivered,96478
1,shipped,1107
2,canceled,625
3,unavailable,609
4,invoiced,314
5,processing,301
6,created,5
7,approved,2


In [ ]:
def timedelta_para_dias(tdelta):
    if pd.isna(tdelta):
        return None
    return tdelta.total_seconds() / (24 * 3600)  # segundos → dias

# Calcular tempos em dias decimais
orders["tempo_compra_para_ordem_dias"] = (orders["order_approved_at"] - orders["order_purchase_timestamp"])
orders["tempo_ordem_para_envio_dias"] = (orders["order_delivered_carrier_date"] - orders["order_approved_at"])
orders["tempo_envio_para_entrega_dias"] = (orders["order_delivered_customer_date"] - orders["order_delivered_carrier_date"])
orders["tempo_entrega_para_estimada_dias"] = (orders["order_delivered_customer_date"] - orders["order_estimated_delivery_date"])
orders["delivery_delay_days"] = (orders["order_delivered_customer_date"] - orders["order_estimated_delivery_date"])

# Mostrar resultado
orders[[
    "tempo_compra_para_ordem_dias",
    "tempo_ordem_para_envio_dias",
    "tempo_envio_para_entrega_dias",
    "tempo_entrega_para_estimada_dias"
]].describe(percentiles=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])


,tempo_compra_para_ordem_dias,tempo_ordem_para_envio_dias,tempo_envio_para_entrega_dias,tempo_entrega_para_estimada_dias
count,99281,97644,96475,96476
mean,0 days 10:25:08.739486910,2 days 19:19:15.298850927,9 days 07:55:59.254148743,-12 days +19:42:04.042383598
std,1 days 02:02:16.813098347,3 days 13:11:10.497083683,8 days 18:14:34.522067763,10 days 04:28:00.181984555
min,0 days 00:00:00,-172 days +18:44:38,-17 days +21:41:31,-147 days +23:36:47
10%,0 days 00:10:05,0 days 10:50:15.100000,1 days 20:38:13.600000,-22 days +17:08:07
20%,0 days 00:12:01,0 days 18:03:39.600000,3 days 06:30:36.400000,-18 days +05:24:51
30%,0 days 00:13:52,0 days 23:54:09.900000,4 days 23:04:04.200000,-16 days +21:08:58
40%,0 days 00:16:18,1 days 07:08:29.200000,6 days 01:47:46.800000,-14 days +18:14:54
50%,0 days 00:20:36,1 days 19:38:29.500000,7 days 02:23:40,-12 days +01:13:31.500000
60%,0 days 00:36:54,2 days 08:54:03.999999999,8 days 06:40:31.800000,-10 days +12:32:48


#### 6 Hypoteses

H1) Orders at the end of the week ate more likely to be delayed

H2) Oders placed during peak seasons have a higher delay rate

H3) Orders shipped closer to holidays have a higher delay rate

H4) Orders placed at night have a higher delay rate than orders placed during day

H5) The avarage order approval time is different on weekday than on weekends

H6) The number of items impact on estimated delivery time